Arranged from [this code](https://github.com/jmtomczak/intro_dgm/blob/main/gans/gans_example.ipynb) by looking at the advices in [here](https://github.com/soumith/ganhacks)

In [1]:
import os

import numpy as np
import matplotlib.pyplot as plt
import torch
from sklearn.datasets import load_digits
from sklearn import datasets
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as tt

In [ ]:
#just playing with hyperparams..

In [2]:
class Digits(Dataset):
    """Scikit-Learn Digits dataset."""

    def __init__(self, mode='train', transforms=None):
        digits = load_digits()
        if mode == 'train':
            self.data = digits.data[:1000].astype(np.float32)
        elif mode == 'val':
            self.data = digits.data[1000:1350].astype(np.float32)
        else:
            self.data = digits.data[1350:].astype(np.float32)

        self.transforms = transforms

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        if self.transforms:
            sample = self.transforms(sample)
        return sample

In [3]:
class Generator(nn.Module):
    def __init__(self, generator_net, z_size):
        super(Generator, self).__init__()

        # We need to init the generator neural net.
        self.generator_net = generator_net
        # We also need to know the size of the latents.
        self.z_size = z_size

    def generate(self, z):
        # Generating for given z is equivalent to applying the neural net.
        return self.generator_net(z)

    def sample(self, batch_size=16):
        # For sampling, we need to sample first latents.
        z = torch.randn(batch_size, self.z_size)
        return self.generate(z)

    def forward(self, z=None):
        if z is None:
            return self.sample()
        else:
            return self.generate(z)

In [4]:
class Discriminator(nn.Module):
    def __init__(self, discriminator_net):
        super(Discriminator, self).__init__()
        # We need to init the discriminator neural net.
        self.discriminator_net = discriminator_net

    def forward(self, x):
        # The forward pass is just about applying the neural net.
        return self.discriminator_net(x)

In [92]:
class GAN(nn.Module):
    def __init__(self, generator, discriminator, EPS=1.e-5):
        super(GAN, self).__init__()

        print('GAN by JT.')

        # To put everything together, we need the generator and
        # the discriminator. NOTE: Both are intanced of classes!
        self.generator = generator
        self.discriminator = discriminator

        # For numerical issue, we introduce a small epsilon.
        self.EPS = EPS

    def forward(self, x_real, reduction='avg', mode='discriminator'):
        # The forward pass calculates the adversarial loss.
        # More specifically, either its part for the generator or
        #  the part for the discriminator.
        if mode == 'discriminator':
            # For the discriminator, we first sample FAKE data.
            x_fake_gen = self.generator.sample(x_real.shape[0])

            # Then, we calculate outputs of the discriminator for the FAKE data.
            # NOTE: We clamp for the numerical stability later on.
            d_fake = torch.clamp(self.discriminator(x_fake_gen), self.EPS, 1. - self.EPS)

            # Moreover, we calculate outputs of the discriminator for the REAL data.
            # NOTE: We clamp for... the numerical stability (again).
            d_real = torch.clamp(self.discriminator(x_real), self.EPS, 1. - self.EPS)

            # The final loss for the discriminator is log(1 - D(G(z))) + log D(x).
            # NOTE: We take the minus sign because we MAXIMIZE the adversarial loss wrt
            # discriminator, so we MINIMIZE the negative adversarial loss wrt discriminator.
            loss = -(torch.log(d_real) + torch.log(1. - d_fake))

        elif mode == 'generator':
            # For the generator, we first sample FAKE data.
            x_fake_gen = self.generator.sample(x_real.shape[0])

            # Then, we calculate outputs of the discriminator for the FAKE data.
            # NOTE: We clamp here for the numerical stability later on.
            d_fake = torch.clamp(self.discriminator(x_fake_gen), self.EPS, 1. - self.EPS)

            # The loss for the generator is log(1 - D(G(z))).
            loss = torch.log(1. - d_fake)



        if reduction == 'sum':
            return loss.sum()
        else:
            return loss.mean()

    def sample(self, batch_size=64):
        return self.generator.sample(batch_size=batch_size)

In [65]:
def evaluation(test_loader, name=None, model_best=None, epoch=None):
    # EVALUATION
    if model_best is None:
        # load best performing model
        model_best = torch.load(name + '.model')

    model_best.eval()
    loss_gen = 0.
    loss_dis = 0.
    N = 0.
    for indx_batch, test_batch in enumerate(test_loader):
        loss_t_gen = model_best.forward(test_batch, reduction='sum', mode='generator')
        loss_t_dis = model_best.forward(test_batch, reduction='sum', mode='discriminator')

        loss_gen = loss_gen + loss_t_gen.item()
        loss_dis = loss_dis + loss_t_dis.item()

        N = N + test_batch.shape[0]

    loss_gen = loss_gen / N
    loss_dis = loss_dis / N

    if epoch is None:
        print(f'FINAL LOSS: gen={loss_gen}, dis={loss_dis}')
    else:
        print(f'Epoch: {epoch}, val gen={loss_gen}, val dis={loss_dis}')

    return loss_gen, loss_dis


def samples_real(name, test_loader):
    # REAL-------
    num_x = 4
    num_y = 4
    x = next(iter(test_loader)).detach().numpy()

    fig, ax = plt.subplots(num_x, num_y)
    for i, ax in enumerate(ax.flatten()):
        plottable_image = np.reshape(x[i], (8, 8))
        ax.imshow(plottable_image, cmap='gray')
        ax.axis('off')

    plt.savefig(name+'_real_images.pdf', bbox_inches='tight')
    plt.close()


def samples_generated(name, data_loader, extra_name=''):
    x = next(iter(data_loader)).detach().numpy()

    # GENERATIONS-------
    model_best = torch.load(name + '.model')
    model_best.eval()

    num_x = 4
    num_y = 4
    x = model_best.sample(num_x * num_y)
    x = x.detach().numpy()

    fig, ax = plt.subplots(num_x, num_y)
    for i, ax in enumerate(ax.flatten()):
        plottable_image = np.reshape(x[i], (8, 8))
        ax.imshow(plottable_image, cmap='gray')
        ax.axis('off')

    plt.savefig(name + '_generated_images' + extra_name + '.pdf', bbox_inches='tight')
    plt.close()


def plot_curve(name, nll_val, xaxis='epochs', yaxis='nll'):
    plt.plot(np.arange(len(nll_val)), nll_val, linewidth='3')
    plt.xlabel(xaxis)
    plt.ylabel(yaxis)
    plt.savefig(name + '_nll_val_curve.pdf', bbox_inches='tight')
    plt.close()

In [66]:
def training(name, num_epochs, model, optimizer_gen, optimizer_dis, training_loader, val_loader):
    gen_val = []
    dis_val = []
    best_loss = 1000.
    patience = 0

    # Main loop
    for e in range(num_epochs):
        # TRAINING
        model.train()
        for indx_batch, batch in enumerate(training_loader):

            # -Discriminator
            loss_dis = model.forward(batch, mode='discriminator')

            optimizer_dis.zero_grad()
            optimizer_gen.zero_grad()
            loss_dis.backward(retain_graph=True)
            optimizer_dis.step()

            # -Generator
            loss_gen = model.forward(batch, mode='generator')

            optimizer_dis.zero_grad()
            optimizer_gen.zero_grad()
            loss_gen.backward(retain_graph=True)
            optimizer_gen.step()

        # Validation
        loss_val_gen, loss_val_dis = evaluation(val_loader, model_best=model, epoch=e)
        gen_val.append(loss_val_gen)  # save for plotting
        dis_val.append(loss_val_dis)  # save for plotting

        torch.save(model, name + '.model')
        samples_generated(name, val_loader, extra_name="_epoch_" + str(e))

    gen_val = np.asarray(gen_val)
    dis_val = np.asarray(dis_val)

    return gen_val, dis_val

In [67]:
transforms_train = tt.Compose( [tt.Lambda(lambda x: 2*(x / 17.)-1.),
                              tt.Lambda(lambda x: torch.from_numpy(x)),
                              tt.Lambda(lambda x: x + 0.03 * torch.randn_like(x))
                              ])

transforms_val  = tt.Compose( [tt.Lambda(lambda x: 2*(x / 17.)-1.),
                               tt.Lambda(lambda x: torch.from_numpy(x)),
                               ])

In [68]:
train_data = Digits(mode='train', transforms=transforms_train)
val_data = Digits(mode='val', transforms=transforms_val)
test_data = Digits(mode='test', transforms=transforms_val)

training_loader = DataLoader(train_data, batch_size=64, shuffle=True)
val_loader = DataLoader(val_data, batch_size=64, shuffle=False)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

In [81]:
D = 64   # input dimension
L = 32 # number of latents
M = 128  # the number of neurons in scale (s) and translation (t) nets

lr_gen = 3e-4 # learning rate
lr_dis = 3e-4 # learning rate
num_epochs = 500 # max. number of epochs

In [70]:
if not(os.path.exists('results')):
    os.mkdir('results')
name = 'gan_' + str(L)
result_dir ='results/' + name + '/'
if not(os.path.exists(result_dir)):
    os.mkdir(result_dir)

In [93]:
# First, we initialize the generator and the discriminator
# -generator
generator_net = nn.Sequential(nn.Linear(L, M), nn.ReLU(),
                              nn.Dropout(0.2),
                              nn.Linear(M, D), nn.Tanh())

generator = Generator(generator_net, z_size=L)

# -discriminator
discriminator_net = nn.Sequential(nn.Linear(D, M), nn.LeakyReLU(),
                                  nn.Linear(M, 1), nn.Sigmoid())

discriminator = Discriminator(discriminator_net)

# Eventually, we initialize the full model
model = GAN(generator=generator, discriminator=discriminator)

GAN by JT.


In [90]:
# OPTIMIZER
optimizer_gen = torch.optim.Adam(model.generator.parameters(), lr=lr_gen)
optimizer_dis = torch.optim.SGD(model.discriminator.parameters(), lr=lr_dis)

In [94]:
# Training procedure
gen_val, dis_val = training(name=result_dir + name,
                            num_epochs=num_epochs,
                            model=model,
                            optimizer_gen=optimizer_gen,
                            optimizer_dis=optimizer_dis,
                            training_loader=training_loader,
                            val_loader=val_loader)

Epoch: 0, val gen=-0.7130560411725726, val dis=1.4113422502790178
Epoch: 1, val gen=-0.712096208844866, val dis=1.4105025046212332
Epoch: 2, val gen=-0.7133825411115374, val dis=1.4125585283551898
Epoch: 3, val gen=-0.7142299488612583, val dis=1.4110513741629465
Epoch: 4, val gen=-0.7119552394321986, val dis=1.4109914725167412
Epoch: 5, val gen=-0.7130860137939453, val dis=1.4113905770438058
Epoch: 6, val gen=-0.710865729195731, val dis=1.4106417628696986
Epoch: 7, val gen=-0.7131664330618722, val dis=1.4113505227225167
Epoch: 8, val gen=-0.7138412475585938, val dis=1.4125099182128906
Epoch: 9, val gen=-0.7142564991542272, val dis=1.4110137939453125
Epoch: 10, val gen=-0.7129605756487165, val dis=1.412064688546317
Epoch: 11, val gen=-0.7120719364711217, val dis=1.4118661390032088
Epoch: 12, val gen=-0.7134599031720843, val dis=1.4124454716273716
Epoch: 13, val gen=-0.7124179458618164, val dis=1.4101186043875558
Epoch: 14, val gen=-0.7132825142996652, val dis=1.4117492239815848
Epoch: 1

In [95]:
test_gen_loss, test_dis_loss = evaluation(name=result_dir + name, test_loader=test_loader)
f = open(result_dir + name + '_test_loss.txt', "w")
f.write(str(test_gen_loss))
f.write('\n')
f.write(str(test_dis_loss))
f.close()

samples_real(result_dir + name, test_loader)

plot_curve(result_dir + name + '_gen', gen_val, yaxis='$log(1-D(G(z))$')
plot_curve(result_dir + name + '_dis', dis_val, yaxis='$-(log(1-D(G(z)) + log(D(x)))$')

samples_generated(result_dir + name, test_loader, extra_name='FINAL')

FINAL LOSS: gen=-0.7128401463997177, dis=1.4039323025901846
